In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import wrds

# -------------------
# USER CONFIG
# -------------------
WRDS_USERNAME = "acaraman"
LIB = "optionm"

TICKER = "AAPL"
START_DATE = "2016-01-01"
END_DATE   = "2025-12-31"

YEARS = list(range(int(START_DATE[:4]), int(END_DATE[:4]) + 1))

# -------------------
# OUTPUT PATHS
# -------------------
RAW_IVYDB_DIR = Path("../../data/raw/ivydb")

# Subdirectories for each data type
VOL_SURFACE_DIR = RAW_IVYDB_DIR / "vol_surface"
SECURITY_PRICE_DIR = RAW_IVYDB_DIR / "security_price"

# Create directories
VOL_SURFACE_DIR.mkdir(parents=True, exist_ok=True)
SECURITY_PRICE_DIR.mkdir(parents=True, exist_ok=True)

print("Output directories:")
print(f"  Vol surface:    {VOL_SURFACE_DIR.resolve()}")
print(f"  Security price: {SECURITY_PRICE_DIR.resolve()}")

Output directories:
  Vol surface:    C:\Users\Admin\OneDrive\Desktop\Fifth Year\Computer Science\CS4490 Thesis\Codebase\Thesis\data\raw\ivydb\vol_surface
  Security price: C:\Users\Admin\OneDrive\Desktop\Fifth Year\Computer Science\CS4490 Thesis\Codebase\Thesis\data\raw\ivydb\security_price


In [ ]:
# Connect to WRDS - may prompt for password
from sqlalchemy import text

try:
    db.close()
except:
    pass

db = wrds.Connection(wrds_username=WRDS_USERNAME)
print(f"Connected to WRDS. Connection type: {type(db)}")

# Get the underlying SQLAlchemy engine for direct queries
engine = db.engine
print(f"Engine type: {type(engine)}")

# Test the connection using native SQLAlchemy execution
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1 AS test"))
    test_result = pd.DataFrame(result.fetchall(), columns=result.keys())
print(f"Connection test: {test_result['test'].iloc[0]} (should be 1)")

Loading library list...
Done
Connected to WRDS. Connection type: <class 'wrds.sql.Connection'>
Engine type: <class 'sqlalchemy.engine.base.Engine'>


C:\Users\Admin\AppData\Local\Temp\ipykernel_27908\542716453.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_result = pd.read_sql(text("SELECT 1 AS test"), conn)


TypeError: Query must be a string unless using sqlalchemy.

In [ ]:
def list_tables(lib=LIB):
    return set(db.list_tables(library=lib))

def describe_cols(table, lib=LIB):
    desc = db.describe_table(library=lib, table=table)
    return desc["name"].tolist(), desc

def pick_col(cols, candidates, required=True):
    cols_l = [c.lower() for c in cols]
    for cand in candidates:
        cand_l = cand.lower()
        if cand_l in cols_l:
            return cols[cols_l.index(cand_l)]
    if required:
        raise KeyError(f"Could not find any of {candidates} in columns: {cols}")
    return None

def has_table(table, lib=LIB):
    return table in list_tables(lib)

def run_query(sql, date_cols=None):
    """Run SQL query using native SQLAlchemy execution."""
    with engine.connect() as conn:
        result = conn.execute(text(sql))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    if date_cols:
        for col in date_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])
    return df

In [ ]:
def resolve_secids_for_ticker(ticker: str, start_date: str, end_date: str, lib=LIB) -> pd.DataFrame:
    """
    Returns a DataFrame with a column 'secid' containing candidate secids for the ticker.
    Tries:
      1) optionm.secnmd with a direct ticker column (ticker/tic/symbol)
      2) optionm.secnmd joined to optionm.ticker (tickerid -> ticker text)
    """
    ticker = ticker.upper()

    if not has_table("secnmd", lib):
        raise RuntimeError(f"{lib}.secnmd not found. Available tables may differ.")

    secnmd_cols, _ = describe_cols("secnmd", lib)

    secid_col = pick_col(secnmd_cols, ["secid"], required=True)
    # try common ticker string columns
    tkr_col = pick_col(secnmd_cols, ["ticker", "tic", "symbol"], required=False)

    # optional effective-date style columns
    eff_col = pick_col(secnmd_cols, ["date", "dt", "effectivedate", "effective_date", "effdate", "eff_date"], required=False)
    end_col = pick_col(secnmd_cols, ["enddate", "end_date", "thrudate", "thru_date"], required=False)

    # Build an overlap filter if effective date ranges exist
    # We want records that overlap [start_date, end_date]
    overlap_filter = ""
    if eff_col and end_col:
        overlap_filter = f"""
          AND {eff_col} <= '{end_date}'
          AND ({end_col} IS NULL OR {end_col} >= '{start_date}')
        """
    elif eff_col:
        overlap_filter = f" AND {eff_col} <= '{end_date}'"

    # Case 1: direct ticker string column exists
    if tkr_col:
        q = f"""
        SELECT DISTINCT {secid_col} AS secid
        FROM {lib}.secnmd
        WHERE {tkr_col} = '{ticker}'
        {overlap_filter}
        """
        out = run_query(q)
        if not out.empty:
            return out

    # Case 2: join through ticker table if secnmd has tickerid
    tickerid_col = pick_col(secnmd_cols, ["tickerid", "ticker_id"], required=False)
    if tickerid_col and has_table("ticker", lib):
        t_cols, _ = describe_cols("ticker", lib)
        t_id = pick_col(t_cols, ["tickerid", "ticker_id"], required=True)
        t_txt = pick_col(t_cols, ["ticker", "tic", "symbol"], required=True)

        q = f"""
        SELECT DISTINCT sn.{secid_col} AS secid
        FROM {lib}.secnmd sn
        JOIN {lib}.ticker t
          ON sn.{tickerid_col} = t.{t_id}
        WHERE t.{t_txt} = '{ticker}'
        {overlap_filter}
        """
        out = run_query(q)
        if not out.empty:
            return out

    raise RuntimeError(
        "Could not resolve secid for ticker. "
        "Inspect optionm.secnmd and optionm.ticker schema to adjust mapping."
    )

secids = resolve_secids_for_ticker(TICKER, START_DATE, END_DATE)
secids

In [ ]:
def choose_best_secid_by_surface_coverage(secids_df: pd.DataFrame, years=YEARS, lib=LIB) -> int:
    """
    If multiple secids exist for a ticker, choose the one with the most distinct surface dates
    across vsurfdYYYY tables within [START_DATE, END_DATE].
    """
    if secids_df.empty:
        raise RuntimeError("No secids provided.")

    secid_list = ",".join(str(int(x)) for x in secids_df["secid"].unique())

    counts = []
    for y in years:
        t = f"vsurfd{y}"
        if not has_table(t, lib):
            continue
        q = f"""
        SELECT secid, COUNT(DISTINCT date) AS n_days
        FROM {lib}.{t}
        WHERE secid IN ({secid_list})
          AND date BETWEEN '{START_DATE}' AND '{END_DATE}'
        GROUP BY secid
        """
        counts.append(run_query(q))

    if not counts:
        raise RuntimeError("No vsurfdYYYY tables found in the selected years.")

    cov = (pd.concat(counts, ignore_index=True)
             .groupby("secid", as_index=False)["n_days"].sum()
             .sort_values("n_days", ascending=False))

    display(cov.head(10))
    return int(cov.iloc[0]["secid"])

SECID = choose_best_secid_by_surface_coverage(secids)
print("Chosen SECID:", SECID)

In [ ]:
def pull_surface_for_secid(secid: int, years=YEARS, lib=LIB) -> pd.DataFrame:
    frames = []
    for y in years:
        t = f"vsurfd{y}"
        if not has_table(t, lib):
            continue

        q = f"""
        SELECT
            secid,
            date,
            days,
            delta,
            cp_flag,
            impl_volatility,
            impl_strike,
            impl_premium,
            dispersion
        FROM {lib}.{t}
        WHERE secid = {secid}
          AND date BETWEEN '{START_DATE}' AND '{END_DATE}'
        """
        frames.append(run_query(q, date_cols=["date"]))

    if not frames:
        raise RuntimeError("No surface data pulled. Check SECID/date range.")

    vs = pd.concat(frames, ignore_index=True)
    return vs

vs = pull_surface_for_secid(SECID)
print("Surface shape:", vs.shape)
vs.head()

In [ ]:
def pull_underlying_prices(secid: int, years=YEARS, lib=LIB) -> pd.DataFrame:
    """
    Tries multiple common IvyDB/WRDS representations:
      1) optionm.security_price if it contains secid
      2) optionm.secprd (often a union/view) if it contains secid
      3) optionm.secprdYYYY tables (yearly)
    Returns a DataFrame with at least ['secid','date', ...]
    """
    # 1) security_price
    if has_table("security_price", lib):
        cols, _ = describe_cols("security_price", lib)
        id_col = pick_col(cols, ["secid", "securityid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)

        if id_col and date_col:
            # If it's keyed by secid, we're done
            if id_col.lower() == "secid":
                q = f"""
                SELECT *
                FROM {lib}.security_price
                WHERE {id_col} = {secid}
                  AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
                """
                return run_query(q, date_cols=["date"])

            # If it's keyed by securityid, we try to map secid -> securityid using optionm.security
            if id_col.lower() == "securityid" and has_table("security", lib):
                s_cols, _ = describe_cols("security", lib)
                s_secid = pick_col(s_cols, ["secid"], required=False)
                s_securityid = pick_col(s_cols, ["securityid"], required=False)

                if s_secid and s_securityid:
                    q_map = f"""
                    SELECT DISTINCT {s_securityid} AS securityid
                    FROM {lib}.security
                    WHERE {s_secid} = {secid}
                    """
                    m = run_query(q_map)
                    if not m.empty:
                        securityid = int(m.iloc[0]["securityid"])
                        q = f"""
                        SELECT *
                        FROM {lib}.security_price
                        WHERE {id_col} = {securityid}
                          AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
                        """
                        out = run_query(q, date_cols=["date"])
                        out["secid"] = secid
                        return out

    # 2) secprd (often a convenient union/view)
    if has_table("secprd", lib):
        cols, _ = describe_cols("secprd", lib)
        id_col = pick_col(cols, ["secid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)
        if id_col and date_col:
            q = f"""
            SELECT *
            FROM {lib}.secprd
            WHERE {id_col} = {secid}
              AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
            """
            return run_query(q, date_cols=["date"])

    # 3) yearly secprdYYYY tables
    frames = []
    for y in years:
        t = f"secprd{y}"
        if not has_table(t, lib):
            continue
        cols, _ = describe_cols(t, lib)
        id_col = pick_col(cols, ["secid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)
        if not (id_col and date_col):
            continue
        q = f"""
        SELECT *
        FROM {lib}.{t}
        WHERE {id_col} = {secid}
          AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
        """
        frames.append(run_query(q, date_cols=["date"]))

    if frames:
        return pd.concat(frames, ignore_index=True)

    raise RuntimeError("Could not pull underlying prices from security_price/secprd/secprdYYYY tables.")

px = pull_underlying_prices(SECID)
print("Price shape:", px.shape)
px.head()

In [ ]:
# Normalize delta: if it's 10..90, convert to 0.10..0.90
if vs["delta"].max() > 2:
    vs["delta"] = vs["delta"] / 100.0

# Cast days to int when safe
vs["days"] = vs["days"].astype(int)

# Ensure cp_flag uppercase
vs["cp_flag"] = vs["cp_flag"].str.upper()

vs[["date","days","delta","cp_flag","impl_volatility","dispersion"]].head()


In [ ]:
print("Surface date range:", vs["date"].min(), "->", vs["date"].max())
print("Unique dates:", vs["date"].nunique())
print("Unique days:", vs["days"].nunique())
print("Unique deltas:", vs["delta"].nunique())
print("cp_flag:", sorted(vs["cp_flag"].unique()))

print("\nDispersion == -99.99 proportion:", (vs["dispersion"] == -99.99).mean())

rows_per_day = vs.groupby("date").size()
print("\nRows per date (describe):")
display(rows_per_day.describe())

# Check grid stability (days x delta x cp)
unique_days = vs["days"].nunique()
unique_deltas = vs["delta"].nunique()
expected_per_day = unique_days * unique_deltas * vs["cp_flag"].nunique()
print("\nIf full grid were present each day, expected rows/day =", expected_per_day)

# A quick missingness view: how many (days,delta,cp) points per day?
grid_points_per_day = vs.groupby(["date"]).apply(lambda g: g[["days","delta","cp_flag"]].drop_duplicates().shape[0])
print("\nDistinct grid points per day (describe):")
display(grid_points_per_day.describe())


In [ ]:
# Save to appropriate subdirectories
vs_out = VOL_SURFACE_DIR / f"{TICKER}_vsurfd_{START_DATE}_{END_DATE}.csv.gz"
px_out = SECURITY_PRICE_DIR / f"{TICKER}_underlying_{START_DATE}_{END_DATE}.csv.gz"

vs.to_csv(vs_out, index=False, compression="gzip")
px.to_csv(px_out, index=False, compression="gzip")

print("Saved surface ->", vs_out)
print("Saved underlying ->", px_out)

In [ ]:
db.close()